In [1]:
import polars as pl

In [2]:
df = pl.scan_parquet('data/nyc_parking_violations_2022.parquet',)

In [7]:
(
    df
        .select(["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"])
        .drop_nulls()
        .limit(10)
        .collect()
)

Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
str,str,str,str,i64
"""NY""","""BUS LANE VIOLA…","""4DSD""","""06/12/2021""",4017749513
"""NJ""","""BUS LANE VIOLA…","""UT""","""06/12/2021""",4017749525
"""NY""","""BUS LANE VIOLA…","""4DSD""","""06/12/2021""",4017749732
"""VA""","""BUS LANE VIOLA…","""SU""","""06/12/2021""",4017749392
"""NY""","""BUS LANE VIOLA…","""SUBN""","""06/12/2021""",4017749951
"""CT""","""BUS LANE VIOLA…","""SD""","""06/12/2021""",4017751180
"""NJ""","""BUS LANE VIOLA…","""WG""","""06/12/2021""",4017751192
"""NY""","""BUS LANE VIOLA…","""TAXI""","""06/12/2021""",4017751209
"""NJ""","""BUS LANE VIOLA…","""PV""","""06/12/2021""",4017751210


In [1]:
(
    df
        .select(["Registration State", "Violation Description"])
        .group_by("Registration State","Violation Description").count()
        .sort("count", descending=True)
        .group_by("Registration State").first()
        .sort("Registration State")
        .collect()
)

Registration State,Violation Description,count
str,str,u32
"""99""",null,17550
"""AB""","""14-No Standing…",22
"""AK""","""PHTO SCHOOL ZN…",125
"""AL""","""PHTO SCHOOL ZN…",3668
"""AR""","""PHTO SCHOOL ZN…",537
"""AZ""","""PHTO SCHOOL ZN…",12188
"""BC""","""14-No Standing…",13
"""CA""","""PHTO SCHOOL ZN…",12994
"""CO""","""PHTO SCHOOL ZN…",2354


In [3]:
%%timeit

(
    df
        .group_by("Vehicle Body Type").agg(Count=pl.count("Summons Number"))
        .sort("Count", descending=True)
        .collect()
)

406 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
